In [1]:
import os
import zipfile

# 1. DOSYA YOLLARINI TANIMLA
zip_path = '/content/drive/MyDrive/Ara Proje/datalar/bus_uclm_separated.zip'
extract_path = '/content/uclm_raw'

# 2. ZİPTEN ÇIKARMA İŞLEMİ
if os.path.exists(zip_path):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)
    print("✅ bus_uclm_separated.zip başarıyla açıldı.\n")
else:
    print(f"❌ Hata: {zip_path} bulunamadı! Lütfen Drive yolunu kontrol et.")

# 3. İÇERİK ANALİZİ (SINIFLARA GÖRE SAYIM)
def count_files_in_subdirs(directory):
    print(f"{'Sınıf Adı':<15} | {'Resim Sayısı'}")
    print("-" * 30)
    total = 0
    # Bazı ziplerde iç içe klasörler olabiliyor, derinlemesine tarıyoruz
    for root, dirs, files in os.walk(directory):
        # Sadece resim dosyalarını say (görünmez sistem dosyalarını ele)
        valid_files = [f for f in files if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
        if valid_files:
            folder_name = os.path.basename(root)
            print(f"{folder_name:<15} | {len(valid_files)}")
            total += len(valid_files)
    print("-" * 30)
    print(f"{'TOPLAM':<15} | {total}")

count_files_in_subdirs(extract_path)

✅ bus_uclm_separated.zip başarıyla açıldı.

Sınıf Adı       | Resim Sayısı
------------------------------
normal          | 419
malign          | 90
benign          | 174
------------------------------
TOPLAM          | 683


In [4]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Yolları yeni yapıya göre güncelledik
base_source = '/content/uclm_raw/bus_uclm_separated'
base_dest = '/content/uclm_tl_baseline'
classes = ['normal', 'benign', 'malign']

# Klasör yapısını oluştur (train/normal, val/normal vb.)
for split in ['train', 'val', 'test']:
    for cls in classes:
        os.makedirs(os.path.join(base_dest, split, cls), exist_ok=True)

# Bölme işlemi
for cls in classes:
    src_dir = os.path.join(base_source, cls)
    # Klasördeki resim listesini al
    files = [f for f in os.listdir(src_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

    # %70 Train, %30 Kalan (Val + Test)
    train_files, temp_files = train_test_split(files, test_size=0.30, random_state=42)
    # Kalan %30'u yarı yarıya böl (%15 Val, %15 Test)
    val_files, test_files = train_test_split(temp_files, test_size=0.50, random_state=42)

    # Dosyaları yeni yapıya kopyala
    for file_list, split in zip([train_files, val_files, test_files], ['train', 'val', 'test']):
        for f in file_list:
            shutil.copy(os.path.join(src_dir, f), os.path.join(base_dest, split, cls, f))

print("✅ Veri seti başarıyla bölündü.")
print(f"📁 Konum: {base_dest}")

✅ Veri seti başarıyla bölündü.
📁 Konum: /content/uclm_tl_baseline


In [6]:
for split in ['train', 'val', 'test']:
    print(f"\n--- {split.upper()} SETİ ---")
    for cls in classes:
        path = os.path.join(base_dest, split, cls)
        print(f"{cls:<10}: {len(os.listdir(path))} resim")


--- TRAIN SETİ ---
normal    : 293 resim
benign    : 121 resim
malign    : 63 resim

--- VAL SETİ ---
normal    : 63 resim
benign    : 26 resim
malign    : 13 resim

--- TEST SETİ ---
normal    : 63 resim
benign    : 27 resim
malign    : 14 resim


In [3]:
import os
print("Mevcut dizin yapısı:")
for root, dirs, files in os.walk('/content/uclm_raw'):
    level = root.replace('/content/uclm_raw', '').count(os.sep)
    indent = ' ' * 4 * (level)
    print(f"{indent}{os.path.basename(root)}/")

Mevcut dizin yapısı:
uclm_raw/
    bus_uclm_separated/
        normal/
        malign/
        benign/


In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
import os

train_path = '/content/uclm_tl_baseline/train'

# Senin belirlediğin hedef katlar ve karşılık gelen yaklaşık sayılar
targets = {
    'normal': 880,  # ~3 katı
    'benign': 725,  # ~6 katı
    'malign': 565   # ~9 katı
}

# Medikal görüntüler için dokuyu bozmayan hafif artırım ayarları
datagen = ImageDataGenerator(
    rotation_range=15,       # Çok çevirmeden
    width_shift_range=0.1,   # Hafif kaydırma
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,          # Hafif yakınlaştırma
    horizontal_flip=True,
    fill_mode='nearest'
)

for cls, target_count in targets.items():
    cls_path = os.path.join(train_path, cls)
    current_files = [f for f in os.listdir(cls_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    num_gen_needed = target_count - len(current_files)

    print(f"🔄 {cls.upper()} artırılıyor: {len(current_files)} -> {target_count} ({num_gen_needed} yeni resim üretilecek)")

    i = 0
    if num_gen_needed > 0:
        while i < num_gen_needed:
            for f in current_files:
                if i >= num_gen_needed: break
                img = load_img(os.path.join(cls_path, f))
                x = img_to_array(img)
                x = x.reshape((1,) + x.shape)

                # Resimleri 'aug_balanced' ön ekiyle kaydet
                for batch in datagen.flow(x, batch_size=1, save_to_dir=cls_path,
                                          save_prefix='aug_balanced', save_format='png'):
                    i += 1
                    break

print("\n✅ Belirlediğin oranlarda artırım tamamlandı!")

🔄 NORMAL artırılıyor: 293 -> 880 (587 yeni resim üretilecek)
🔄 BENIGN artırılıyor: 121 -> 725 (604 yeni resim üretilecek)
🔄 MALIGN artırılıyor: 63 -> 565 (502 yeni resim üretilecek)

✅ Belirlediğin oranlarda artırım tamamlandı!


In [8]:
import zipfile
import os

# Ziplenecek klasör ve kaydedilecek yol
folder_to_zip = '/content/uclm_tl_baseline'
output_zip_path = '/content/drive/MyDrive/Ara Proje/datalar/uclm_tl_baseline_augmented.zip'

def zip_folder(folder_path, output_path):
    with zipfile.ZipFile(output_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                # Klasör yapısını koruyarak ziple
                zipf.write(os.path.join(root, file),
                           os.path.relpath(os.path.join(root, file), folder_path))

print("📦 Zipleme işlemi başlıyor... (Bu işlem veri boyutuna göre birkaç dakika sürebilir)")
zip_folder(folder_to_zip, output_zip_path)
print(f"✅ İşlem tamamlandı! Dosya şuraya kaydedildi: {output_zip_path}")

📦 Zipleme işlemi başlıyor... (Bu işlem veri boyutuna göre birkaç dakika sürebilir)
✅ İşlem tamamlandı! Dosya şuraya kaydedildi: /content/drive/MyDrive/Ara Proje/datalar/uclm_tl_baseline_augmented.zip


In [9]:
def check_final_counts(base_path):
    classes = ['normal', 'benign', 'malign']
    print(f"\n{'Bölüm':<10} | {'Sınıf':<10} | {'Resim Sayısı'}")
    print("-" * 35)

    for split in ['train', 'val', 'test']:
        for cls in classes:
            path = os.path.join(base_path, split, cls)
            if os.path.exists(path):
                count = len([f for f in os.listdir(path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))])
                print(f"{split:<10} | {cls:<10} | {count}")
        print("-" * 35)

check_final_counts('/content/uclm_tl_baseline')


Bölüm      | Sınıf      | Resim Sayısı
-----------------------------------
train      | normal     | 865
train      | benign     | 709
train      | malign     | 547
-----------------------------------
val        | normal     | 63
val        | benign     | 26
val        | malign     | 13
-----------------------------------
test       | normal     | 63
test       | benign     | 27
test       | malign     | 14
-----------------------------------


In [12]:
import os
import shutil
from sklearn.model_selection import train_test_split

# YOLLAR
source_path = '/content/uclm_raw/bus_uclm_separated'
dest_30 = '/content/Uclm_da_30'
classes = ['normal', 'benign', 'malign']

# Klasörleri temizle ve oluştur
if os.path.exists(dest_30):
    shutil.rmtree(dest_30)

for split in ['train', 'val', 'test']:
    for cls in classes:
        os.makedirs(os.path.join(dest_30, split, cls), exist_ok=True)

for cls in classes:
    src_dir = os.path.join(source_path, cls)
    files = [f for f in os.listdir(src_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

    # MALIGN sınıfı için özel oran, diğerleri için hocanın %30 oranı
    if cls == 'malign':
        # 90 resmin 41'i Train (~%45), kalanı Test/Val
        train_files, temp_files = train_test_split(files, train_size=41, random_state=42)
    else:
        # %30 Train, %70 Kalan
        train_files, temp_files = train_test_split(files, test_size=0.70, random_state=42)

    # Kalan veriyi Val ve Test olarak bölüyoruz
    # Malign için: 49 kalan resmin 9'u Val, 40'ı Test olacak şekilde
    val_size = 9 if cls == 'malign' else 0.15 # Oransal veya sabit
    val_files, test_files = train_test_split(temp_files, test_size=0.82, random_state=42)

    for file_list, split in zip([train_files, val_files, test_files], ['train', 'val', 'test']):
        for f in file_list:
            shutil.copy(os.path.join(src_dir, f), os.path.join(dest_30, split, cls, f))

print("✅ Malign sınıfı optimize edilerek %30-70 bölme işlemi bitti.")

✅ Malign sınıfı optimize edilerek %30-70 bölme işlemi bitti.


In [19]:
import os
import shutil
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from sklearn.model_selection import train_test_split

# YOLLAR
source_path = '/content/uclm_raw/bus_uclm_separated'
master_pool = '/content/UCLM_Master_Pool'
classes = ['normal', 'benign', 'malign']
targets = {'normal': 1000, 'benign': 750, 'malign': 450}

# Klasörleri Hazırla
if os.path.exists(master_pool): shutil.rmtree(master_pool)
for cls in classes: os.makedirs(os.path.join(master_pool, cls), exist_ok=True)

datagen = ImageDataGenerator(rotation_range=20, width_shift_range=0.1, height_shift_range=0.1,
                             shear_range=0.1, zoom_range=0.1, horizontal_flip=True, fill_mode='nearest')

print("📦 Master Havuz oluşturuluyor...")

for cls in classes:
    src_dir = os.path.join(source_path, cls)
    files = [f for f in os.listdir(src_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

    # Önce orijinalleri kopyala
    for f in files:
        shutil.copy(os.path.join(src_dir, f), os.path.join(master_pool, cls, f))

    # Hedef sayıya ulaşana kadar artır
    current_count = len(os.listdir(os.path.join(master_pool, cls)))
    needed = targets[cls] - current_count

    if needed > 0:
        i = 0
        while i < needed:
            for f in files:
                if i >= needed: break
                img = load_img(os.path.join(src_dir, f))
                x = img_to_array(img).reshape((1,) + img_to_array(img).shape)
                for batch in datagen.flow(x, batch_size=1, save_to_dir=os.path.join(master_pool, cls),
                                          save_prefix='aug_master', save_format='png'):
                    i += 1
                    break
    print(f"✅ {cls.upper()}: {len(os.listdir(os.path.join(master_pool, cls)))} resim hazır.")

📦 Master Havuz oluşturuluyor...
✅ NORMAL: 983 resim hazır.
✅ BENIGN: 737 resim hazır.
✅ MALIGN: 444 resim hazır.


In [20]:
def create_final_scenarios(train_ratio, scenario_name):
    base_dest = f'/content/UCLM_Final_Scenarios/{scenario_name}'
    for split in ['train', 'test']:
        for cls in classes: os.makedirs(os.path.join(base_dest, split, cls), exist_ok=True)

    for cls in classes:
        src_dir = os.path.join(master_pool, cls)
        all_files = os.listdir(src_dir)

        # Bölme İşlemi
        train_f, test_f = train_test_split(all_files, train_size=train_ratio, random_state=42)

        for f in train_f: shutil.copy(os.path.join(src_dir, f), os.path.join(base_dest, 'train', cls, f))
        for f in test_f: shutil.copy(os.path.join(src_dir, f), os.path.join(base_dest, 'test', cls, f))

    print(f"📂 {scenario_name} (%{int(train_ratio*100)}) Klasörü Tamamlandı.")

# Senaryoları Uygula
create_final_scenarios(0.30, "otuz")
create_final_scenarios(0.50, "elli")

📂 otuz (%30) Klasörü Tamamlandı.
📂 elli (%50) Klasörü Tamamlandı.


In [22]:
import zipfile
import os

# Ziplenecek ana klasör (otuz ve elli klasörlerini içeren dizin)
folder_to_zip = '/content/UCLM_Final_Scenarios'

# Drive yolu
drive_folder = '/content/drive/MyDrive/Ara Proje/datalar/'
output_zip_path = os.path.join(drive_folder, 'Uclm_Scenarios_Final.zip')

# Klasör yoksa oluştur
if not os.path.exists(drive_folder):
    os.makedirs(drive_folder)

print(f"📦 Zipleme işlemi başladı... (Hedef: {output_zip_path})")

def zip_folder(folder_path, output_path):
    with zipfile.ZipFile(output_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                # Klasör yapısını (otuz/train/normal vb.) koruyarak ziple
                zipf.write(os.path.join(root, file),
                           os.path.relpath(os.path.join(root, file), folder_path))

zip_folder(folder_to_zip, output_zip_path)

print(f"✅ İşlem tamamlandı! 'otuz' ve 'elli' senaryoları tek zip olarak kaydedildi.")

📦 Zipleme işlemi başladı... (Hedef: /content/drive/MyDrive/Ara Proje/datalar/Uclm_Scenarios_Final.zip)
✅ İşlem tamamlandı! 'otuz' ve 'elli' senaryoları tek zip olarak kaydedildi.


In [23]:
import os
import zipfile

# Dosya yolu
zip_path = '/content/drive/MyDrive/Ara Proje/datalar/Uclm_for_DA.zip'

if os.path.exists(zip_path):
    print(f"✅ Dosya bulundu: {zip_path}")
    print("\n📦 Zip İçeriği ve Sayı Kontrolü:")
    print("-" * 45)

    # Zip dosyasını açmadan içeriğini oku
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        file_list = zip_ref.namelist()

        # Sayımları tutmak için sözlük
        counts = {}

        for file in file_list:
            # Klasörleri atla, sadece dosyalara bak
            if not file.endswith('/'):
                # Yol yapısı: senaryo/split/sınıf/dosya.png
                parts = file.split('/')
                if len(parts) >= 3:
                    senaryo = parts[0]
                    split = parts[1]
                    sinif = parts[2]

                    key = f"{senaryo} | {split} | {sinif}"
                    counts[key] = counts.get(key, 0) + 1

        # Sonuçları yazdır
        print(f"{'Senaryo':<8} | {'Bölüm':<6} | {'Sınıf':<8} | {'Adet'}")
        print("-" * 45)
        for key in sorted(counts.keys()):
            sen, spl, sin = key.split(' | ')
            print(f"{sen:<8} | {spl:<6} | {sin:<8} | {counts[key]}")

else:
    print(f"❌ Hata: Dosya belirtilen yolda bulunamadı!")

✅ Dosya bulundu: /content/drive/MyDrive/Ara Proje/datalar/Uclm_for_DA.zip

📦 Zip İçeriği ve Sayı Kontrolü:
---------------------------------------------
Senaryo  | Bölüm  | Sınıf    | Adet
---------------------------------------------
elli     | test   | benign   | 369
elli     | test   | malign   | 222
elli     | test   | normal   | 492
elli     | train  | benign   | 368
elli     | train  | malign   | 222
elli     | train  | normal   | 491
otuz     | test   | benign   | 516
otuz     | test   | malign   | 311
otuz     | test   | normal   | 689
otuz     | train  | benign   | 221
otuz     | train  | malign   | 133
otuz     | train  | normal   | 294
